# آٹو جن ٹول استعمال کی مثال


## ضروری پیکجز درآمد کریں


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## کلائنٹ بنانا

اس نمونے میں، ہم [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) کا استعمال کریں گے تاکہ LLM تک رسائی حاصل کی جا سکے۔

`model` کو `gpt-4o-mini` کے طور پر بیان کیا گیا ہے۔ مختلف نتائج دیکھنے کے لیے ماڈل کو GitHub Models مارکیٹ پلیس میں دستیاب کسی اور ماڈل میں تبدیل کرنے کی کوشش کریں۔

ایک فوری ٹیسٹ کے طور پر، ہم ایک سادہ پرامپٹ چلائیں گے - `What is the capital of France`۔


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## فنکشنز کی تعریف

اس مثال میں، ہم ایجنٹ کو ایک ایسے ٹول تک رسائی دیں گے جو ایک فنکشن ہے جس میں دستیاب چھٹیوں کے مقامات اور ان کی دستیابی کی فہرست موجود ہے۔

آپ یہ تصور کر سکتے ہیں کہ یہ ایک ایسا منظر ہوگا جہاں ایک ٹریول ایجنٹ کے پاس، مثال کے طور پر، ایک سفری ڈیٹا بیس تک رسائی ہو سکتی ہے۔

جب آپ اس نمونے کو دیکھیں، تو بلا جھجک نئے فنکشنز اور ٹولز کی تعریف کرنے کی کوشش کریں جنہیں ایجنٹ کال کر سکے۔


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## فنکشن ٹول کی تعریف  
ایجنٹ کو `vacation_destinations` کو بطور `FunctionTool` استعمال کرنے کے لیے، ہمیں اسے ایک فنکشن ٹول کے طور پر تعریف کرنا ہوگا۔  

ہم اس ٹول کی وضاحت بھی فراہم کریں گے جو ایجنٹ کے لیے یہ سمجھنے میں مددگار ہوگی کہ یہ ٹول صارف کی درخواست کردہ کام کے حوالے سے کس مقصد کے لیے استعمال ہوتا ہے۔  


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## ایجنٹ کی تعریف

اب ہم نیچے دیے گئے کوڈ میں ایجنٹ بنا سکتے ہیں۔ ہم `system_message` کو اس لیے متعین کرتے ہیں تاکہ ایجنٹ کو یہ ہدایات دی جا سکیں کہ وہ صارفین کو چھٹیوں کی منزلیں تلاش کرنے میں کیسے مدد کرے۔

ہم `reflect_on_tool_use` پیرامیٹر کو بھی true پر سیٹ کرتے ہیں۔ یہ LLM کو اس قابل بناتا ہے کہ وہ ٹول کال کے جواب کو لے اور اسے قدرتی زبان میں بھیجے۔

آپ اس پیرامیٹر کو false پر سیٹ کر سکتے ہیں تاکہ فرق دیکھ سکیں۔


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## ایجنٹ کو چلانا

اب ہم ایجنٹ کو ابتدائی صارف پیغام کے ساتھ چلا سکتے ہیں جس میں ٹوکیو کا سفر کرنے کی درخواست کی گئی ہے۔

آپ اس شہر کی منزل کو تبدیل کر سکتے ہیں تاکہ دیکھ سکیں کہ ایجنٹ شہر کی دستیابی پر کیسے ردعمل دیتا ہے۔


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا عدم درستگی ہو سکتی ہیں۔ اصل دستاویز، جو اس کی مقامی زبان میں ہے، کو مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے لیے ہم ذمہ دار نہیں ہیں۔
